In [1]:
import torch
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data.dataset import Dataset, TensorDataset
from torch.utils.data.dataloader import DataLoader
import json
import pickle
import string
from DiacriticDataset import DiacriticDataset

In [2]:
train_dataset = DiacriticDataset('Dataset/train_cleaned_noshadda_215.txt','Dataset/letter_to_id.pickle','Dataset/id_to_letter.pickle','Dataset/diacritic_to_id.pickle','Dataset/id_to_diacritic.pickle','Dataset/word_to_id.pickle','Dataset/id_to_word.pickle')

In [4]:
train_dataloader = DataLoader(train_dataset)

In [11]:
class AutoDiacriticNet(nn.Module):
    def __init__(self,word_emb_dim, hidden_dim, letter_vocab_size,word_vocab_size):
        super(AutoDiacriticNet, self).__init__()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.letter_embedding = nn.Embedding(letter_vocab_size, word_emb_dim)
        self.lstm = nn.LSTM(input_size=word_emb_dim,hidden_size=hidden_dim,num_layers=2,bidirectional=True,batch_first=True)
        
    def forward(self,letters,words):
        model_input = letters.to(self.device)
        embedded = self.letter_embedding(model_input)
        lstm_out, _ = self.lstm(embedded)
        output = lstm_out
        return output
        

In [12]:
EPOCHS = 1
LETTER_EMBEDDING_DIM = 5
HIDDEN_DIM = 256

letter_vocab_size = len(train_dataset.letter_to_id)


use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

model = AutoDiacriticNet(LETTER_EMBEDDING_DIM,HIDDEN_DIM,letter_vocab_size,None)

if use_cuda:
    model.cuda()
    
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
acumulate_grad_steps = 64

In [13]:
for epoch in range(EPOCHS):
    i = 0
    for batch_idx, input_data in enumerate(train_dataloader):
        i += 1
        letters = input_data[0]
        diacritics = input_data[1]
        print(model(letters,None).size())
        break

torch.Size([1, 129, 512])
